# 功課三 - Create a NB Classifier
* use the dictionary created from HW2
* 13 classes (id 1~13), each class has 15 training documents (195)
* calculate M for each class
    * calculate with add-one smoothing   
* feature selection - 500 terms 

## Import

In [1]:
# import nltk
# import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

# nltk.download('stopwords')

In [2]:
import numpy as np
import pandas as pd
from collections import Counter
import pickle
import math
import glob
import os

In [3]:
from nltk.stem import PorterStemmer
p = PorterStemmer()
stemmed_doc =[]

In [4]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

## Create train test

In [5]:
os.chdir('../Data/IRTM/')
my_files = glob.glob('*.txt')

# /Users/stephanie/Desktop/2022_Information_Retrieve/HW3

In [6]:
my_dict={}

punctuations = '''!()-[]{};:'`"\,<>./?@#$%^&*_~'''
numbers ='1234567890'

for file in my_files:

    with open(file) as f:
        lines = f.read()
    file_num = file.strip('.txt')
    
    # 處理標點符號與大小寫, 句子沒有斷乾淨
    for punc in punctuations:
        lines = lines.replace(punc, ' ').replace('\n','')
    for num in numbers:
        lines = lines.replace(num,'')
        
    doc_list = lines.lower().split(' ')
    
    stemmed_doc = [p.stem(token) for token in doc_list if p.stem(token)!= '\n' and not p.stem(token).isnumeric() and len(p.stem(token))>1 ]
    
    
    final_tokens=[]
    for word in stemmed_doc:
        if word not in final_tokens and word not in stop_words and not word.isnumeric() and len(word)>1:
            final_tokens.append(word)

    my_dict[file_num] ={}
    my_dict[file_num]['tokens'] = final_tokens
    my_dict[file_num]['label'] = ''
    
    # for token in final_tokens:
    #     if token in my_dict:
    #         my_dict[token]+=1
    #     else:
    #         my_dict[token]=1

In [7]:
len(my_dict)

1095

In [8]:
os.chdir('..')
os.chdir('..')

text =[]
file_path = './HW3/training.txt'
with open(file_path) as f:
    lines = f.read()
    text.append(lines)
print(text)


['1 11 19 29 113 115 169 278 301 316 317 321 324 325 338 341 \n2 1 2 3 4 5 6 7 8 9 10 12 13 14 15 16 \n3 813 817 818 819 820 821 822 824 825 826 828 829 830 832 833 \n4 635 680 683 702 704 705 706 708 709 719 720 722 723 724 726 \n5 646 751 781 794 798 799 801 812 815 823 831 839 840 841 842 \n6 995 998 999 1003 1005 1006 1007 1009 1011 1012 1013 1014 1015 1016 1019 \n7 700 730 731 732 733 735 740 744 752 754 755 756 757 759 760 \n8 262 296 304 308 337 397 401 443 445 450 466 480 513 533 534 \n9 130 131 132 133 134 135 136 137 138 139 140 141 142 143 145 \n10 31 44 70 83 86 92 100 102 305 309 315 320 326 327 328 \n11 240 241 243 244 245 248 250 254 255 256 258 260 275 279 295 \n12 535 542 571 573 574 575 576 578 581 582 583 584 585 586 588 \n13 485 520 523 526 527 529 530 531 532 536 537 538 539 540 541 ']


In [9]:
train_dict = {}
test_dict = {}

# train
for i in text[0].split('\n'):
    cnt=0
    for j in i.split(' '):
        if len(j)>0:
            if cnt == 0 :
                cnt+=1
                label = j
            else:
                my_dict[j]['label']=label
                train_dict[j]=my_dict[j]

#test 
for i in my_dict:
    if i not in train_dict:
        test_dict[i]=my_dict[i]


In [10]:
print(len(train_dict) , len(test_dict))

195 900


In [11]:
train_dict

{'11': {'tokens': ['navi',
   'unit',
   'state',
   'japan',
   'south',
   'korea',
   'singapor',
   'monday',
   'began',
   'first',
   'combin',
   'submarin',
   'rescu',
   'exercis',
   'pacif',
   'day',
   'reach',
   'involv',
   'peopl',
   'four',
   'ship',
   'three',
   'sophist',
   'underwat',
   'devic',
   'personnel',
   'distress',
   'joint',
   'china',
   'sea',
   'wa',
   'plan',
   'nearli',
   'two',
   'year',
   'befor',
   'recent',
   'russian',
   'disast',
   'kill',
   'particip',
   'said',
   'heighten',
   'import',
   'thi',
   'time',
   'realli',
   'chanc',
   'work',
   'togeth',
   'region',
   'lt',
   'lesli',
   'hull',
   'ryde',
   'spokeswoman',
   'promot',
   'greater',
   'understand',
   'also',
   'commit',
   'stabil',
   'asia',
   'ha',
   'begun',
   'ventur',
   'shallow',
   'coastal',
   'water',
   'use',
   'test',
   'capabl',
   'capt',
   'leidig',
   'deputi',
   'coordin',
   'look',
   'oper',
   'make',
   'transi

In [12]:
import operator
train_df = pd.DataFrame.from_dict(train_dict)
test_df = pd.DataFrame.from_dict(test_dict)

In [21]:
train_df = train_df.T
test_df = test_df.T

In [22]:
test_df

,tokens,label
1053,"[earthquak, struck, thi, citi, wa, fill, voic,...",
1047,"[india, devast, earthquak, offici, believ, may...",
289,"[year, old, yemeni, boy, say, man, paid, watch...",
504,"[biggest, race, young, career, olymp, judi, ma...",
1090,"[india, tonight, sever, aftershock, quak, zone...",
...,...,...
1089,"[death, natur, western, india, still, find, su...",
1062,"[two, week, deadli, earthquak, el, salvador, d...",
290,"[thi, world, lisa, mullin, yemeni, coastlin, s...",
284,"[investig, widen, probe, bomb, uss, cole, saud...",


#

In [14]:
# cols = ['t_index' , 'term' , 'df']
# df = df[cols]

In [15]:
# df.to_csv(r'c:\data\pandas.txt', header=None, index=None, sep=' ', mode='a')

## Transfer to unit vector

In [16]:
# for file in my_files:
    
#     unit_df = pd.DataFrame(columns=['t_index' , 'tf_idf'])
    
#     with open(file) as f:
#         lines = f.read()
    
#     # 處理標點符號與大小寫, 句子沒有斷乾淨
#     for punc in punctuations:
#         lines = lines.replace(punc, ' ').replace('\n','')
#     for num in numbers:
#         lines = lines.replace(num,'')
        
#     doc_list = lines.lower().split(' ')
    
#     stemmed_doc = [p.stem(token) for token in doc_list if p.stem(token)!= '\n' and not p.stem(token).isnumeric() and len(p.stem(token))>1 ]
    
    
#     final_tokens=[]
#     for word in stemmed_doc:
#         if word not in stop_words and not word.isnumeric() and len(word)>1:
#             final_tokens.append(word)
        
#     N = len(my_files)
#     # create the tf-idf vector
#     for tok in final_tokens:
#         tf = final_tokens.count(tok)/ len(final_tokens)
#         unit_df.loc[len(unit_df)]=[df[df.term == tok].t_index.to_list()[0] , math.log(float(N) / df[df.term == tok].df.to_list()[0]) ]
# #     print(unit_df)
#     unit_df.to_csv('/home/emma/bilab/Steph_C/IR/doc'+file, header=None, index=None, sep=' ', mode='a')

## Cosine Similarity Function

In [17]:
# from numpy import dot
# from numpy.linalg import norm

In [18]:
# def cosine(doc_x , doc_y):
    
#     # read files
#     x = pd.read_csv(doc_x, sep=" ", header=None)
#     y = pd.read_csv(doc_y, sep=" ", header=None)
#     x.columns = ['t_index' , 'tf_idf']
#     y.columns = ['t_index' , 'tf_idf']
    
#     # get bag of words
#     bag_of_words = set(x.t_index.to_list()+y.t_index.to_list())
    
#     x_list =[]
#     y_list =[]
    
#     # calculate cosine similarity
#     for word in bag_of_words:
        
#         if word in x.t_index.to_list():
#             x_list.append(x[x.t_index == word].tf_idf.to_list()[0])
#         elif word not in x.t_index.to_list():
#             x_list.append(0)
        
#         if word in y.t_index.to_list():
#             y_list.append(y[y.t_index == word].tf_idf.to_list()[0])
#         elif word not in y.t_index.to_list():
#             y_list.append(0)
            
    
#     return  dot(x_list, y_list)/(norm(x_list)*norm(y_list))
    


In [19]:
# cosine('/home/emma/bilab/Steph_C/IR/doc1.txt','/home/emma/bilab/Steph_C/IR/doc2.txt')